<a href="https://colab.research.google.com/github/mouktik05/research/blob/main/mouktik_cdna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install biopython requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 7.9 MB/s eta 0:00:00


In [ ]:
import requests

In [ ]:
def get_transcript_exons(transcript_id):
    """Fetch exon information for a given Ensembl transcript ID."""
    #url = f"https://rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    ##url = f"https://rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    url = f"https://grch37.rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    #https://grch37.rest.ensembl.org/overlap/id/ENST00000025008?feature=gene;content-type=application/json
    #https://rest.ensembl.org/overlap/id/ENST00000025008?feature=gene;content-type=application/json
    #response = requests.get(url, headers={"Content-Type": "application/json"})
    response = requests.get(url, json={"start": "value"})
    print("url",url)
    if response.status_code != 200:
        raise Exception(f"API request failed with status code {response.status_code}")
    return response.json()

def map_cdna_to_genomic(transcript_id, cdna_position):
    """Map a cDNA position to a genomic coordinate using Ensembl."""
    exons = get_transcript_exons(transcript_id)
    total_cdna_len = 0
    for exon in sorted(exons, key=lambda x: x['start']):
        exon_cdna_start = total_cdna_len + 1
        exon_cdna_end = total_cdna_len + exon['end'] - exon['start'] + 1
        total_cdna_len = exon_cdna_end

        if exon_cdna_start <= cdna_position <= exon_cdna_end:
            genomic_pos = exon['start'] + (cdna_position - exon_cdna_start)
            return exon['seq_region_name'], genomic_pos, exon['strand']

    raise ValueError(f"cDNA position {cdna_position} out of range for transcript {transcript_id}")


In [ ]:

transcript_id = "ENST00000050961"  # Replace with your transcript ID for USP28
cdna_position = 5374  # Replace with your cDNA position
chromosome, genomic_position, strand = map_cdna_to_genomic(transcript_id, cdna_position)
print(f"Chromosome: {chromosome}, Genomic Position: {genomic_position}, Strand: {strand}")



url https://grch37.rest.ensembl.org/overlap/id/ENST00000050961?feature=gene;content-type=application/json
Chromosome: 8, Genomic Position: 77408808, Strand: -1


In [ ]:
# prompt: in the below cell, ignore exception in with loop and continue with other records

transcript_id = "ENST00000003302"  # Replace with your transcript ID for USP28
cdna_position = 2194  # Replace with your cDNA position

try:
    chromosome, genomic_position, strand = map_cdna_to_genomic(transcript_id, cdna_position)
    print(f"Chromosome: {chromosome}, Genomic Position: {genomic_position}, Strand: {strand}")
except Exception as e:
    print(f"Error processing transcript {transcript_id}: {e}")
    # Continue with other records
    pass


In [ ]:
# prompt: python program to read from a CSV, extract column coding_region_effect and write to another file
import re
import csv
import requests

def get_transcript_exons(transcript_id):
    """Fetch exon information for a given Ensembl transcript ID."""
    ##url = f"https://rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    ##url = f"https://rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    url = f"https://grch37.rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    #https://grch37.rest.ensembl.org/overlap/id/ENST00000025008?feature=gene;content-type=application/json
    #https://rest.ensembl.org/overlap/id/ENST00000025008?feature=gene;content-type=application/json
    #response = requests.get(url, headers={"Content-Type": "application/json"})
    response = requests.get(url, json={"start": "value"})
    print("url",url)
    if response.status_code != 200:
        raise Exception(f"API request failed with status code {response.status_code}")
    return response.json()

def map_cdna_to_genomic(transcript_id, cdna_position):
    """Map a cDNA position to a genomic coordinate using Ensembl."""
    exons = get_transcript_exons(transcript_id)
    total_cdna_len = 0
    for exon in sorted(exons, key=lambda x: x['start']):
        exon_cdna_start = total_cdna_len + 1
        exon_cdna_end = total_cdna_len + exon['end'] - exon['start'] + 1
        total_cdna_len = exon_cdna_end

        if exon_cdna_start <= cdna_position <= exon_cdna_end:
            genomic_pos = exon['start'] + (cdna_position - exon_cdna_start)
            return exon['seq_region_name'], genomic_pos, exon['strand']

    raise ValueError(f"cDNA position {cdna_position} out of range for transcript {transcript_id}")


def parse_variant(variant_str):
    # Define a regular expression pattern to extract information
    pattern = r'^(ENST\d+)\((\w+)\):c\.(-?\d+)([ACGTNacgtn])>([ACGTNacgtn])$'

    # Use regex to match the pattern in the variant string
    match = re.match(pattern, variant_str)

    if match:
        enst = match.group(1)  # ENST ID
        gene_name = match.group(2)  # Gene name
        position = match.group(3)  # Position
        ref_base = match.group(4).upper()  # Reference base
        alt_base = match.group(5).upper()  # Alternate base

        return enst, gene_name, position, ref_base, alt_base
    else:
        return 'NA', 'NA', 'NA', 'NA', 'NA'

# Open the input CSV file
with open('odbfile.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)

    # Open the output file for writing
    with open('output37.csv', 'w', newline='') as output_file:
        csv_writer = csv.writer(output_file)

        # Write the header row
        csv_writer.writerow(['MRN','coding_region_effect','enst','gene_name','position','ref_base','alt_base','chromosome', 'genomic_position', 'strand'])

        # Iterate through the input rows
        for row in csv_reader:
            # Extract the 'coding_region_effect' column value
            coding_region_effect = row[5]
            mrn = row[0]
            enst, gene_name, position, ref_base, alt_base = parse_variant(coding_region_effect)





            if enst != 'NA':
              print("position value is ", position)
              try:
                chromosome, genomic_position, strand = map_cdna_to_genomic(enst, int(position))
                # Write the coding region effect value to the output file
                csv_writer.writerow([mrn,coding_region_effect,enst,gene_name,position,ref_base,alt_base,chromosome, genomic_position, strand])

              except Exception as e:
                print(f"Error processing transcript {enst}: {e}")
                # Continue with other records
                pass

            else:
              chromosome = 'NA'
              genomic_position = 'NA'
              strand = 'NA'


            # Write the coding region effect value to the output file
            ##csv_writer.writerow([mrn,coding_region_effect,enst,gene_name,position,ref_base,alt_base,chromosome, genomic_position, strand])

In [ ]:
# prompt: python program to read from a CSV, extract column coding_region_effect and write to another file
import re
import csv
import requests

def get_transcript_exons(transcript_id):
    """Fetch exon information for a given Ensembl transcript ID."""
    ##url = f"https://rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    url = f"https://rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    ##url = f"https://grch37.rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    #https://grch37.rest.ensembl.org/overlap/id/ENST00000025008?feature=gene;content-type=application/json
    #https://rest.ensembl.org/overlap/id/ENST00000025008?feature=gene;content-type=application/json
    #response = requests.get(url, headers={"Content-Type": "application/json"})
    response = requests.get(url, json={"start": "value"})
    print("url",url)
    if response.status_code != 200:
        raise Exception(f"API request failed with status code {response.status_code}")
    return response.json()

def map_cdna_to_genomic(transcript_id, cdna_position):
    """Map a cDNA position to a genomic coordinate using Ensembl."""
    exons = get_transcript_exons(transcript_id)
    total_cdna_len = 0
    for exon in sorted(exons, key=lambda x: x['start']):
        exon_cdna_start = total_cdna_len + 1
        exon_cdna_end = total_cdna_len + exon['end'] - exon['start'] + 1
        total_cdna_len = exon_cdna_end

        if exon_cdna_start <= cdna_position <= exon_cdna_end:
            genomic_pos = exon['start'] + (cdna_position - exon_cdna_start)
            return exon['seq_region_name'], genomic_pos, exon['strand']

    raise ValueError(f"cDNA position {cdna_position} out of range for transcript {transcript_id}")


def parse_variant(variant_str):
    # Define a regular expression pattern to extract information
    pattern = r'^(ENST\d+)\((\w+)\):c\.(-?\d+)([ACGTNacgtn])>([ACGTNacgtn])$'

    # Use regex to match the pattern in the variant string
    match = re.match(pattern, variant_str)

    if match:
        enst = match.group(1)  # ENST ID
        gene_name = match.group(2)  # Gene name
        position = match.group(3)  # Position
        ref_base = match.group(4).upper()  # Reference base
        alt_base = match.group(5).upper()  # Alternate base

        return enst, gene_name, position, ref_base, alt_base
    else:
        return 'NA', 'NA', 'NA', 'NA', 'NA'

# Open the input CSV file
with open('odbfile.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)

    # Open the output file for writing
    with open('output38.csv', 'w', newline='') as output_file:
        csv_writer = csv.writer(output_file)

        # Write the header row
        csv_writer.writerow(['MRN','coding_region_effect','enst','gene_name','position','ref_base','alt_base','chromosome', 'genomic_position', 'strand'])

        # Iterate through the input rows
        for row in csv_reader:
            # Extract the 'coding_region_effect' column value
            coding_region_effect = row[5]
            mrn = row[0]
            enst, gene_name, position, ref_base, alt_base = parse_variant(coding_region_effect)





            if enst != 'NA':
              print("position value is ", position)
              try:
                chromosome, genomic_position, strand = map_cdna_to_genomic(enst, int(position))

              except Exception as e:
                print(f"Error processing transcript {enst}: {e}")
                # Continue with other records
                pass

            else:
              chromosome = 'NA'
              genomic_position = 'NA'
              strand = 'NA'


            # Write the coding region effect value to the output file
            csv_writer.writerow([mrn,coding_region_effect,enst,gene_name,position,ref_base,alt_base,chromosome, genomic_position, strand])

In [ ]:
# prompt: python program to read from a CSV, extract column coding_region_effect and write to another file
## Extracting Chromosome, Alt, Ref, and Strand from specific MRNS
import re
import csv
import requests

def get_transcript_exons(transcript_id):
    """Fetch exon information for a given Ensembl transcript ID."""
    ##url = f"https://rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    url = f"https://rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    ##url = f"https://grch37.rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    #https://grch37.rest.ensembl.org/overlap/id/ENST00000025008?feature=gene;content-type=application/json
    #https://rest.ensembl.org/overlap/id/ENST00000025008?feature=gene;content-type=application/json
    #response = requests.get(url, headers={"Content-Type": "application/json"})
    response = requests.get(url, json={"start": "value"})
    print("url",url)
    if response.status_code != 200:
        raise Exception(f"API request failed with status code {response.status_code}")
    return response.json()

def map_cdna_to_genomic(transcript_id, cdna_position):
    """Map a cDNA position to a genomic coordinate using Ensembl."""
    exons = get_transcript_exons(transcript_id)
    total_cdna_len = 0
    for exon in sorted(exons, key=lambda x: x['start']):
        exon_cdna_start = total_cdna_len + 1
        exon_cdna_end = total_cdna_len + exon['end'] - exon['start'] + 1
        total_cdna_len = exon_cdna_end

        if exon_cdna_start <= cdna_position <= exon_cdna_end:
            genomic_pos = exon['start'] + (cdna_position - exon_cdna_start)
            return exon['seq_region_name'], genomic_pos, exon['strand']

    raise ValueError(f"cDNA position {cdna_position} out of range for transcript {transcript_id}")


def parse_variant(variant_str):
    # Define a regular expression pattern to extract information
    pattern = r'^(ENST\d+)\((\w+)\):c\.(-?\d+)([ACGTNacgtn])>([ACGTNacgtn])$'

    # Use regex to match the pattern in the variant string
    match = re.match(pattern, variant_str)

    if match:
        enst = match.group(1)  # ENST ID
        gene_name = match.group(2)  # Gene name
        position = match.group(3)  # Position
        ref_base = match.group(4).upper()  # Reference base
        alt_base = match.group(5).upper()  # Alternate base

        return enst, gene_name, position, ref_base, alt_base
    else:
        return 'NA', 'NA', 'NA', 'NA', 'NA'

# Open the input CSV file
with open('odbfile.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)

    # Open the output file for writing
    with open('output38.csv', 'w', newline='') as output_file:
        csv_writer = csv.writer(output_file)

        # Write the header row
        csv_writer.writerow(['MRN','coding_region_effect','enst','gene_name','position','ref_base','alt_base','chromosome', 'genomic_position', 'strand'])

        # Iterate through the input rows
        for row in csv_reader:
            # Extract the 'coding_region_effect' column value
            coding_region_effect = row[5]
            mrn = row[0]
            enst, gene_name, position, ref_base, alt_base = parse_variant(coding_region_effect)





            if enst != 'NA':
              print("position value is ", position)
              try:
                chromosome, genomic_position, strand = map_cdna_to_genomic(enst, int(position))

              except Exception as e:
                print(f"Error processing transcript {enst}: {e}")
                # Continue with other records
                pass

            else:
              chromosome = 'NA'
              genomic_position = 'NA'
              strand = 'NA'


            # Write the coding region effect value to the output file
            csv_writer.writerow([mrn,coding_region_effect,enst,gene_name,position,ref_base,alt_base,chromosome, genomic_position, strand])

In [ ]:
import pandas as pd
import os

# Define the input CSV file path and output directory
input_csv_file = 'output_hg37.csv'  # Change this to your input file path
output_directory = 'mrn_files'  # Change this to your desired output directory

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Read the input CSV file
df = pd.read_csv(input_csv_file)

# Drop the columns 'Chromosome', 'Genomic Position', 'Strand'
df_dropped = df.drop(columns= ['coding_region_effect', 'enst', 'gene_name'])

# Group the DataFrame by the 'MRN' column
grouped = df_dropped.groupby('MRN')

# Iterate over each group and save to a separate .dat file
for mrn, group_df in grouped:
    output_file = os.path.join(output_directory, f'{mrn}.dat')
    # Save the DataFrame to a .dat file with tab separation
    group_df.to_csv(output_file, sep='\t', index=False)
    print(f'Saved {output_file}')


In [1]:
import pandas as pd
import os

# Define the input CSV file path and output directory
input_csv_file = 'output_hg37.csv'  # Change this to your input file path
output_directory = 'mrn_files'  # Change this to your desired output directory

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Read the input CSV file
df = pd.read_csv(input_csv_file)

# Define columns to remove
columns_to_remove = ['coding_region_effect', 'enst', 'gene_name', 'position']

# Remove specified columns
df_filtered = df.drop(columns=columns_to_remove)

# Group the DataFrame by the 'MRN' column
grouped = df_filtered.groupby('MRN')

# Function to format DataFrame for .dat file
def format_dataframe_for_dat(df):
    # Determine maximum column widths based on header length
    col_widths = {col: len(col) for col in df.columns}

    # Update column widths with data lengths
    for col in df.columns:
        col_widths[col] = max(col_widths[col], df[col].astype(str).apply(len).max())

    # Format each row
    formatted_rows = []
    for _, row in df.iterrows():
        formatted_row = " ".join(f"{str(val).ljust(col_widths[col])}" for col, val in row.items())
        formatted_rows.append(formatted_row)

    # Combine header and formatted rows
    header_row = " ".join(f"{col.ljust(col_widths[col])}" for col in df.columns)
    return "\n".join([header_row] + formatted_rows)

# Iterate over each group and save to a separate .dat file
for mrn, group_df in grouped:
    output_file = os.path.join(output_directory, f'{mrn}.dat')
    formatted_data = format_dataframe_for_dat(group_df)

    with open(output_file, 'w') as f:
        f.write(formatted_data)

    print(f'Saved {output_file}')



FileNotFoundError: [Errno 2] No such file or directory: 'output_hg37.csv'

In [5]:
pip install biopython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 19.4 MB/s eta 0:00:00


In [4]:
pip install pyfaidx


In [11]:
import os
import pandas as pd
from Bio.Seq import Seq
from pyfaidx import Fasta

# Load the reference genome
 #genome = Fasta('/path/to/your/hg19.fa')

genome = Fasta('401428_mouktik.fasta')

# Function to get the sequence context and apply reverse complement if necessary
def get_ms(chr, pos, strand, alt_allele, sep=" "):
    # Get the sequence context (3 bp upstream and downstream)
    seq = get_ms_gr(chr, pos)
    print("seq", seq)
    # Convert to Biopython Seq object
    seq = Seq(str(seq))
    alt_seq = Seq(alt_allele)

    # Apply reverse complement if necessary
    reverse_mask = any(nucleotide in seq for nucleotide in ["A", "T", "C", "G"])
    if reverse_mask:
        seq = seq.reverse_complement()
        alt_seq = alt_seq.reverse_complement()

    # Create the result
    result = f"{seq}{sep}{alt_seq}"
    return result

# Function to get the sequence context (3 bp upstream and downstream)
def get_ms_gr(chr, pos):
    # Fetch the 7bp sequence centered around the position
    start = pos - 3
    end = pos + 3
    seq = genome[chr][start:end].seq
    return seq

# Set working directory
os.chdir(".")

# Get list of files
file_names = [f for f in os.listdir() if f.endswith('.txt')]

# Process each file
for j, file_name in enumerate(file_names):
    print(file_name)
    print(j + 1)

    # Load the data
    data = pd.read_csv(file_name, sep=" ", names=["chr", "loc", "site", "mut", "strand"])

    # Filter data by chromosome
    for chr_num in range(1, 23):
        data_chr = data[data['chr'] == chr_num]

        # Process each location in the filtered data
        for i in data_chr['loc']:
            tri_nucl_context = get_ms(f"chr{chr_num}", i, "+", "C")
            df = pd.DataFrame({"loc": [i], "triNuclContext": [tri_nucl_context]})

            # Save the result to a file
            output_file = f"/Users/4464689/Desktop/PROJECTS/Moffitt/GhulamIPMNs/Mouktik/sequences3nt/{file_name}"
            df.to_csv(output_file, sep="\t", mode='a', header=False, index=False)

    print(f"Processed chr{chr_num}")



In [8]:
def convert_dat_to_fasta(dat_file, fasta_file):
    with open(dat_file, 'r') as infile, open(fasta_file, 'w') as outfile:
        sequence_id = 1  # Starting sequence ID number
        for line in infile:
            line = line.strip()
            if line:  # Ignore empty lines
                # If the line contains sequence data, write it in FASTA format
                outfile.write(f">sequence{sequence_id}\n")
                outfile.write(f"{line}\n")
                sequence_id += 1

# Example usage
convert_dat_to_fasta('401428.dat', '401428_mouktik.fasta')
